# Steel Defects

Problem Statement: Based on the dataset provided by Severstal through Kaggle.

Objectives:
* practice computer vision machine learning models
* practice model deployment with AWS

In [9]:
import os
import fastbook
from fastbook import *

import pandas as pd
import numpy as np

TO-DO LIST:
* data extraction
* data cleaning
* feature transformation
* target transformation
* baseline
* model selection
* hyperparameter optimization
* export model

# 1. Data Extraction

Data will be extracted using the kaggle API.

In [2]:
#!kaggle competitions download -c severstal-steel-defect-detection
#comment out if used in an environment with active repo

In [3]:
train_ims = './serverstal-steel-defect-detection/train_images'
test_ims = './serverstal-steel-defect-detection/test_images'

In [4]:
train = get_image_files(train_ims)
test = get_image_files(test_ims)

In [5]:
train

(#12568) [Path('serverstal-steel-defect-detection/train_images/0002cc93b.jpg'),Path('serverstal-steel-defect-detection/train_images/00031f466.jpg'),Path('serverstal-steel-defect-detection/train_images/000418bfc.jpg'),Path('serverstal-steel-defect-detection/train_images/000789191.jpg'),Path('serverstal-steel-defect-detection/train_images/0007a71bf.jpg'),Path('serverstal-steel-defect-detection/train_images/000a4bcdd.jpg'),Path('serverstal-steel-defect-detection/train_images/000f6bf48.jpg'),Path('serverstal-steel-defect-detection/train_images/0014fce06.jpg'),Path('serverstal-steel-defect-detection/train_images/001982b08.jpg'),Path('serverstal-steel-defect-detection/train_images/001d1b355.jpg')...]

In [6]:
test

(#5506) [Path('serverstal-steel-defect-detection/test_images/0000f269f.jpg'),Path('serverstal-steel-defect-detection/test_images/000ccc2ac.jpg'),Path('serverstal-steel-defect-detection/test_images/002451917.jpg'),Path('serverstal-steel-defect-detection/test_images/003c5da97.jpg'),Path('serverstal-steel-defect-detection/test_images/0042e163f.jpg'),Path('serverstal-steel-defect-detection/test_images/004f40c73.jpg'),Path('serverstal-steel-defect-detection/test_images/00513039a.jpg'),Path('serverstal-steel-defect-detection/test_images/006f39c41.jpg'),Path('serverstal-steel-defect-detection/test_images/008725cbc.jpg'),Path('serverstal-steel-defect-detection/test_images/0098ca44e.jpg')...]

# 2. Initial Clean-Up

In [7]:
failed = verify_images(train) # verifies if no empty path

In [8]:
failed # returns which indexes are empty paths

(#0) []

# 3. Preparation

Goal: the way we know there is a defect for each image is through the `train.csv` file. Every row contains the ID and the defect, meaning that each ID can have multiple rows if it has multiple defects. 

Therefore, if we want to know that there IS a defect as opposed to identifying WHAT defect, we want to determine if an ID on this list exists. 

In [28]:
train_csv = pd.read_csv('serverstal-steel-defect-detection/train.csv')
train_csv.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 30370 24 30626 24 30882 24 31139 23 31395 23 31651 23 31907 23 32163 23 32419 23 32675 23 77918 27 78174 55 78429 60 78685 64 78941 68 79197 72 79452 77 79708 81 79964 85 80220 89 80475 94 80731 98 80987 102 81242 105 81498 105 81754 104 82010 104 82265 105 82521 31 82556 69 82779 27 82818 63 83038 22 83080 57 83297 17 83342 50 83555 13 83604 44 83814 8 83866 37 84073 3 84128 31 84390 25 84652 18 84918 8 85239 10 85476 29 85714 47 85960 57 86216 57 86471 58 86727 58 86983 58 87238 59 87494 59 87750 59 88005 60 88261 60 88517 60 88772 61 89028 53...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 110 19859 110 20115 110 20371 110 20627 110 20883 110 21139 110 21395 110 21651 110 21962 55 293125 251 293381 251 293637 251 293893 251 294149 251 294405 251 294661 251 294917 251 295173 251 295429 251 295685 251 295941 251 296197 251 296453 251 296709 251 296965 251 297221 251 297477 251 297733 251 297989 251 298245 251 298564 188 298945 63
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 38863 28 39119 28 39375 29 39631 29 39887 29 40143 29 40399 29 40655 30 40911 30 41167 30 41423 30 41679 31 41935 31 42191 31 42447 31 42703 31 42960 31 43216 31 43472 31 43728 31 43984 31 44240 32 44496 32 44752 32 45008 32 45264 33 45520 33 45776 33 46032 33 46288 33 46544 34 46803 31 47065 25 47327 19 47588 15 47850 9 48112 3 62667 12 62923 23 63179 23 63348 3 63435 23 63604 7 63691 23 63860 11 63947 23 64116 15 64203 23 64372 19 64459 23 64628 24 64715 23 64884 28 64971 23 65139 33 65227 23 65395 37 65483 23 65651 41 65740 22 65907 45 65996 22...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 133248 13 133503 16 133757 19 134012 22 134267 24 134522 26 134777 29 135032 31 135287 34 135542 36 135796 40 136050 43 136304 46 136558 50 136812 54 137066 56 137320 59 137574 61 137828 63 138082 65 138336 68 138590 70 138845 71 139101 71 139356 73 139612 73 139868 73 140123 74 140379 74 140634 75 140890 75 141145 77 141400 78 141654 80 141909 81 142164 82 142418 84 142673 85 142928 86 143182 88 143437 89 143692 90 143946 93 144201 94 144456 95 144710 97 144965 98 145220 99 145474 101 145729 103 145983 105 146237 107 146491 109 146745 112 1469...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468 92 230623 10 230724 95 230845 11 230869 28 230979 97 231094 63 231235 97 231344 70 231490 99 231593 79 231746 183 232002 184 232257 186 232513 186 232769 185 233024 186 233280 185 233535 186 233791 185 234047 185 234302 186 234558 186 234813 186 235069 186 235327 184 235586 181 235845 177 236103 175 236362 172 236621 78 236714 71 236880 57 236972 61 237139 44 237229 52 237397 32 237486 11 237513 16 237656 19 237744 3 237915 6


In [ ]:
# goal is to rename a file so that it has a 0 if it has a defect at the front and a 1 at the front if it does have a defect
# for path in train, 
# if the picture id image is in ImageID column
# rename picture path


In [31]:
# for path in train:
# """
# Renames the files in train_images folder with a 1 if it
# HAS a defect (at least one) and 0 if it does not. 
# Determined if the file name is in the folder. ONLY RUN THIS ONCE.
# """
#     ID = path.name
#     if ID in train_csv['ImageId'].unique(): # which means it has a defect
#         path.replace('serverstal-steel-defect-detection/train_images/' +'1_' + ID) #1 is has defect
#     else:
#         path.replace('serverstal-steel-defect-detection/test_images/' + '0_' + ID)#0 is no defect

In [36]:
train1 = get_image_files(train_ims)
train1

(#6666) [Path('serverstal-steel-defect-detection/train_images/1_0002cc93b.jpg'),Path('serverstal-steel-defect-detection/train_images/1_0007a71bf.jpg'),Path('serverstal-steel-defect-detection/train_images/1_000a4bcdd.jpg'),Path('serverstal-steel-defect-detection/train_images/1_000f6bf48.jpg'),Path('serverstal-steel-defect-detection/train_images/1_0014fce06.jpg'),Path('serverstal-steel-defect-detection/train_images/1_0025bde0c.jpg'),Path('serverstal-steel-defect-detection/train_images/1_002af848d.jpg'),Path('serverstal-steel-defect-detection/train_images/1_002fc4e19.jpg'),Path('serverstal-steel-defect-detection/train_images/1_0030401a5.jpg'),Path('serverstal-steel-defect-detection/train_images/1_0046839bd.jpg')...]

# 4. Feature Transformation

Create the dataloader to pass the image data onto the GPU for training.

In [ ]:
steel_pics = DataBlock(
    blocks = (ImageBlock, CategoryBlock), # i.e., Image = Independent variable, Category = Dependent Variable
    get_items = get_image_files,
    spliter = RandomSplitter(valid_pct = 0.2, seed = 42), # for splitting validation and training set
    get_y = lambda x: 
    
)

### To-Do:

1. Data extraction
2. Data cleaning
3. Preparation (change file names to be useful)
4. Feature Transformation
5. Scaling (dataloaders?)
6. Target Transformation
7. Model Selection
8. Hyperparameter Optimization
9. Ensembling

In [32]:
train[1]

Path('serverstal-steel-defect-detection/train_images/00031f466.jpg')

In [33]:
train[2]

Path('serverstal-steel-defect-detection/train_images/000418bfc.jpg')